## Predicciones Modelo de Pytorch

In [1]:
import torch

In [2]:
# carga de pesos
checkpoint = torch.load("model_checkpoint.pth")

In [3]:
# carga del vocabulario

vocab = torch.load("vocab.pt")

/Users/diegocardona/repositorios/NLP_nequi_test_diegocardona/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from torchtext.data.utils import get_tokenizer

tokenizador = get_tokenizer("basic_english")

texto_pipeline = lambda x: vocab(tokenizador(x))

In [9]:
num_class = 5
vocab_size = 10000
embedding_size = 20
TASA_APRENDIZAJE = 0.15

In [10]:
from torch import nn
import torch.nn.functional as F

class ModeloClasificacionTexto(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(ModeloClasificacionTexto, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.bn1 = nn.BatchNorm1d(embed_dim)
        self.fc = nn.Linear(embed_dim, num_class)

    def forward(self, text_batch):
        embedded = self.embedding(text_batch)  # (batch_size, seq_len, embed_dim)
        # promedio por secuencia (ignorando padding opcional)
        embedded = embedded.mean(dim=1)  # (batch_size, embed_dim)
        embedded_norm = self.bn1(embedded)
        activated = F.relu(embedded_norm)
        return self.fc(activated)

In [11]:
model = ModeloClasificacionTexto(vocab_size=vocab_size, embed_dim=embedding_size, num_class=num_class)

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=TASA_APRENDIZAJE)

model.load_state_dict(checkpoint["model_state_dict"])

<All keys matched successfully>

In [13]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

In [14]:
category_mapping = {
    0: 'Credit Reporting',
    1: 'Debt Collection',
    2: 'Loans',
    3: 'Credit Card Services',
    4: 'Bank Accounts and Services'
}

In [15]:
ejemplo_inferencia = "I have a problem with my credit report"

In [30]:
def predict(text, texto_pipeline):
  model.eval()
  with torch.no_grad():
    text_ids = texto_pipeline(text)
    text_tensor = torch.tensor([text_ids], dtype=torch.long)
    text_tensor = text_tensor.to(next(model.parameters()).device)
    # opt_mod = torch.compile(model, mode="reduce-overhead") # aumenta el uso de memoria pero optimiza la inferencia, para modelos grandes max-autotune
    # output = opt_mod(text, torch.tensor([0]))
    output = model(text_tensor)
  return output.argmax(1).item()
  # return output

In [24]:
model_cpu = model.to("cpu")

In [25]:
category_mapping[predict(ejemplo_inferencia, texto_pipeline)]

'Loans'

In [29]:
print(predict(ejemplo_inferencia, texto_pipeline))

tensor([[-29.3394,  -7.8461,  12.9861, -13.4300,  -8.4167]])
